# Project Anomaly Finder API Example

### This Jupyter notebook shows you how to get started with the Project Anomaly Finder API in Python, and how to visualize your results.

In [1]:
import requests
import json
import pandas as pd
import numpy as np
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
from bokeh.plotting import figure,output_notebook, show
from bokeh.palettes import Blues4
from bokeh.models import ColumnDataSource,Slider
import datetime
from bokeh.io import push_notebook
from dateutil import parser
from ipywidgets import interact, widgets, fixed
output_notebook()

Loading BokehJS ...

In [3]:
def detect(endpoint, subscription_key, request_data):
    headers = {'Content-Type': 'application/json', 'Ocp-Apim-Subscription-Key': subscription_key}
    response = requests.post(endpoint, data=json.dumps(request_data), headers=headers)
    if response.status_code == 200:
        return json.loads(response.content.decode("utf-8"))
    else:
        print(response.status_code)
        raise Exception(response.text)

In [4]:
def build_figure(result, sample_data, sensitivity):
    columns = {'ExpectedValues': result['ExpectedValues'], 'IsAnomaly': result['IsAnomaly'], 'IsNegativeAnomaly': result['IsNegativeAnomaly'],
              'IsPositiveAnomaly': result['IsPositiveAnomaly'], 'UpperMargins': result['UpperMargins'], 'LowerMargins': result['LowerMargins']
              , 'Value': [x['Value'] for x in sample_data['Series']], 'Timestamp': [parser.parse(x['Timestamp']) for x in sample_data['Series']]}
    response = pd.DataFrame(data=columns)
    values = response['Value']
    label = response['Timestamp']
    anomalies = []
    anomaly_labels = []
    index = 0
    anomaly_indexes = []
    p = figure(x_axis_type='datetime', title="Anomaly Finder Result", width=800, height=600)
    for anom in response['IsAnomaly']:
        if anom == True and (values[index] > response.iloc[index]['ExpectedValues'] + response.iloc[index]['UpperMargins'] or 
                         values[index] < response.iloc[index]['ExpectedValues'] - response.iloc[index]['LowerMargins']):
            anomalies.append(values[index])
            anomaly_labels.append(label[index])
            anomaly_indexes.append(index)
        index = index+1
    upperband = response['ExpectedValues'] + response['UpperMargins']
    lowerband = response['ExpectedValues'] -response['LowerMargins']
    band_x = np.append(label, label[::-1])
    band_y = np.append(lowerband, upperband[::-1])
    boundary = p.patch(band_x, band_y, color=Blues4[2], fill_alpha=0.5, line_width=1, legend='Boundary')
    p.line(label, values, legend='Value', color="#2222aa", line_width=1)
    p.line(label, response['ExpectedValues'], legend='ExpectedValue',  line_width=1, line_dash="dotdash", line_color='olivedrab')
    anom_source = ColumnDataSource(dict(x=anomaly_labels, y=anomalies))
    anoms = p.circle('x', 'y', size=5, color='tomato', source=anom_source)
    p.legend.border_line_width = 1
    p.legend.background_fill_alpha  = 0.1
    show(p, notebook_handle=True)

### Detect latest anomaly of sample timeseries

Find anomalies in batch will detect all points with same model, this method can not adapt to series pattern's change well. Usually, for online monitoring scenario, user can choose to detect latest one with a limited window size series, in the belowing example, we simulte a streaming scenario, each time we detect the latest one with 29 points before. (since period of the series is 7).

In [5]:
# Variables
endpoint_latest = 'https://westus2.api.cognitive.microsoft.com/anomalyfinder/v2.0/timeseries/last/detect'
subscription_key = '' #Here you have to paste your primary key

In [6]:
def detect_anomaly(sensitivity):
    sample_data = json.load(open('sample.json'))
    points = sample_data['Series']
    skip_point = 29
    result = {'ExpectedValues': [None]*len(points), 'UpperMargins': [None]*len(points), 
              'LowerMargins': [None]*len(points), 'IsNegativeAnomaly': [False]*len(points), 
              'IsPositiveAnomaly':[False]*len(points), 'IsAnomaly': [False]*len(points)}
    anom_count = 0
    for i in range(skip_point, len(points)+1):
        single_sample_data = {}
        single_sample_data['Series'] = points[i-29:i]
        single_sample_data['Granularity'] = 'daily'
        single_sample_data['MaxAnomalyRatio'] = 0.25
        single_sample_data['Sensitivity'] = sensitivity
        single_point = detect(endpoint_latest, subscription_key, single_sample_data)
        if single_point['IsAnomaly'] == True:
            anom_count = anom_count + 1

        result['ExpectedValues'][i-1] = single_point['ExpectedValue']
        result['UpperMargins'][i-1] = single_point['UpperMargin']
        result['LowerMargins'][i-1] = single_point['LowerMargin']
        result['IsNegativeAnomaly'][i-1] = single_point['IsNegativeAnomaly']
        result['IsPositiveAnomaly'][i-1] = single_point['IsPositiveAnomaly']
        result['IsAnomaly'][i-1] = single_point['IsAnomaly']
    
    build_figure(result, sample_data, sensitivity)

In [7]:
# 95 sensitvity
detect_anomaly(95)

In [8]:
# 85 sensitvity
detect_anomaly(85)